In [19]:
sc

<SparkContext master=local[*] appName=PySparkShell>

### 1. read points 

In [20]:
# Load one dataset 
pointsPath="hdfs:////user/ds503/project3/Points.csv"
points=sc.textFile(pointsPath)

### 2. define two helper function

In [21]:
# Input point x, y
# Output cell ID
def getCellID(x, y):
    cell_x = int((x-1) / 20) + 1 
    cell_y = int((y-1) / 20) + 1 
    return cell_x + (500 - cell_y) * 500

In [22]:
# Input cell ID
# Output cell neighbour ID array
def getNeighbour(c):
    upLeft = c - 501
    up = c - 500
    upRight = c - 499

    left = c - 1
    right = c + 1

    lowLeft = c + 499
    low = c + 500
    lowRight = c + 501
    
    if(c % 500 == 1):
        res = [up, upRight, right, low, lowRight]
        res = list(filter(lambda x: (x >= 1) & (x <= 250000), res))
    elif(c % 500 == 0):
        res = [upLeft, up, left, lowLeft, low]
        res = list(filter(lambda x: (x >= 1) & (x <= 250000), res))
    else:
        res = [upLeft, up, upRight, left, right, lowLeft, low, lowRight]
        res = list(filter(lambda x: (x >= 1) & (x <= 250000), res))
    return res
    


### 3. parse point
from string to tuple (x, y)

In [23]:
pointsXY = points.map(lambda x: [int(x.split(",")[0]), int(x.split(",")[1])])
pointsXY.take(2)

[[2535, 8149], [155, 6776]]

### 4. compute cell points count 
(cellID, pointCount)

In [24]:
cellPointsCount = pointsXY.map(lambda x: (getCellID(x[0], x[1]), 1))\
                          .reduceByKey(lambda x, y: x + y)
cellPointsCount.take(2)

[(80508, 46), (36938, 49)]

### 5. compute cell neighbour count
(cellID, neighbourCount)

In [25]:
def mapFunc1(x):
    cellID = x[0]
    cellNeigCount = len(getNeighbour(cellID))
    return [cellID, cellNeigCount]   

cellNeigCount = cellPointsCount.map(mapFunc1)
cellNeigCount.take(2)

[[80508, 8], [36938, 8]]

### 6. compute cell neighbour point avg count
(cellID, neighbourAvgPointCount)

In [26]:
def mapFunc2(x):
    cellID = x[0]
    cellNeigID = getNeighbour(cellID)
    for oneNeig in cellNeigID:
        yield (oneNeig, cellID)
        
cellNeigFlatCount = cellPointsCount.flatMap(mapFunc2).leftOuterJoin(cellPointsCount)
cellNeigFlatCount.take(2)

[(80008, (80508, 41)), (80008, (79508, 41))]

In [27]:
# (cellID, neighbourPointCount)
def mapFunc3(x):
    cellID = x[1][0]
    cellNeigPointCount = 0 if(x[1][1] == None) else x[1][1]
    return (cellID, cellNeigPointCount)
cellNeigPointCount = cellNeigFlatCount.map(mapFunc3).reduceByKey(lambda x, y: x + y)
cellNeigPointCount.take(2)

[(80508, 363), (79508, 327)]

In [28]:
cellNeigAvgCount = cellNeigPointCount.join(cellNeigCount)\
                                     .map(lambda x: (x[0], (x[1][0]/x[1][1])))
cellNeigAvgCount.take(2)

[(80508, 45.375), (228372, 40.25)]

### 7. compute cell density

In [29]:
cellDensity = cellPointsCount.join(cellNeigAvgCount)\
                             .map(lambda x: (x[0], x[1][0]/ x[1][1]))\
                             .sortBy(lambda x: -x[1])


In [30]:
cellDensity.take(3)

[(44947, 2.032154340836013),
 (68303, 1.8313253012048192),
 (46501, 1.8206521739130437)]

### 8. Top10 Density

In [31]:
top10 = sc.parallelize(cellDensity.take(10))

In [32]:
top10.collect()

[(44947, 2.032154340836013),
 (68303, 1.8313253012048192),
 (46501, 1.8206521739130437),
 (241132, 1.7969230769230768),
 (104811, 1.7655172413793103),
 (6457, 1.7610062893081762),
 (55493, 1.7476923076923077),
 (226717, 1.741046831955923),
 (84024, 1.7379310344827585),
 (142898, 1.732484076433121)]

### 9. TOP k grid cells w.r.t their Relative-Density Scores
(cellID, cellDensity, numOfNeighbours, neighbourID, neighbourDensity)

In [33]:
def flatMapFunc4(x):
    cellNeig = getNeighbour(x[0])
    for oneNeig in cellNeig:
        yield (oneNeig, (x[0], x[1], len(cellNeig)))

def mapFunc5(x):
    neigID = x[0]
    cellID = x[1][0][0]
    cellDensity = x[1][0][1]
    neigCount = x[1][0][2]
    neigDensity = x[1][1]
    
    return cellID, cellDensity, neigCount, neigID, neigDensity
    
rdd = top10.flatMap(flatMapFunc4)\
           .leftOuterJoin(cellDensity)\
           .map(mapFunc5)\
           .sortBy(lambda x: x[0])


In [34]:
rdd.collect()

[(6457, 1.7610062893081762, 8, 6456, 0.8467966573816156),
 (6457, 1.7610062893081762, 8, 6458, 0.8085106382978723),
 (6457, 1.7610062893081762, 8, 5956, 0.8465608465608465),
 (6457, 1.7610062893081762, 8, 5957, 0.7466666666666667),
 (6457, 1.7610062893081762, 8, 5958, 0.8541666666666666),
 (6457, 1.7610062893081762, 8, 6956, 0.6736842105263158),
 (6457, 1.7610062893081762, 8, 6957, 1.0386740331491713),
 (6457, 1.7610062893081762, 8, 6958, 1.0162162162162163),
 (44947, 2.032154340836013, 8, 44448, 0.6253369272237197),
 (44947, 2.032154340836013, 8, 45446, 0.9824561403508771),
 (44947, 2.032154340836013, 8, 45447, 0.7091412742382271),
 (44947, 2.032154340836013, 8, 45448, 0.6772486772486772),
 (44947, 2.032154340836013, 8, 44946, 0.7411444141689373),
 (44947, 2.032154340836013, 8, 44948, 1.0823529411764705),
 (44947, 2.032154340836013, 8, 44446, 1.1685393258426966),
 (44947, 2.032154340836013, 8, 44447, 0.9859943977591037),
 (46501, 1.8206521739130437, 5, 46502, 1.032258064516129),
 (465